In [ ]:
# Unzip the fruit images
!unzip fruits.zip

In [ ]:
# Clean up our folder and filenames
rm -rf `find -type d -name .ipynb_checkpoints`
%cd fruits
%mv apple_validation.jpg apple_test.jpg
%mv banana_validation.png banana_test.png
%mv tomato_validation.jpg tomato_test.jpg
%cd ..

In [ ]:
# Load our images into a NumPy array called "training_data"
import cv2
import os
import random
import matplotlib.pyplot as plt

IMG_SIZE = 100
dir="fruits/data"

def get_folder_names(directory_path):
    folder_names = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]
    return folder_names


categories = get_folder_names(dir)

def get_img_data(directory):

  img_data = []
  for catago in categories:
      folder = os.path.join(directory, catago)
      label = categories.index(catago)
      for img in os.listdir(folder):
          img_path = os.path.join(folder, img)
          try:
              # Attempt to read and resize the image
              img_array = cv2.imread(img_path)
              # img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
              # img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

              new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

              # Check if the image array is not empty
              if new_array is not None and not new_array.size == 0:
                  img_data.append([new_array, label])
              else:
                  print(f"Error loading or resizing image: {img_path}")

          except Exception as e:
              print(f"Error processing image {img_path}: {str(e)}")
  return img_data

training_data = get_img_data(dir)

img_index = 0
# Note what order the fruit categories are printed in. You will need to reference
# this order during the model prediction step (the very last cell in this notebook)
print(categories)
print(len(training_data))

## You can uncomment these lines to understand how NumPy visualizes our images

# print(training_data[img_index][1])
# plt.imshow(training_data[img_index][0], cmap = "gray")
# plt.axis('off')
# plt.show()

# print(training_data[img_index])
# print(training_data[img_index][0].shape)

In [ ]:
# Shuffle our array of images (training_data) to mix up
# our CNN's learning pattern
import random
random.shuffle(training_data)

In [ ]:
# Separate our image data into two different arrays
# 1. "features" is the image data our CNN will be trained on
# 2. "labels" is what the CNN will use to classify each image (i.e. Apple, Orange, etc.)
import numpy as np
x=[]
y=[]
for features,labels in training_data:
    x.append(features)
    y.append(labels)

#Convert X and Y list into array
X=np.array(x).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y=np.array(y)

# Normalization
X = X/255

from tensorflow.keras.utils import to_categorical
# Since we're training the CNN to only identify 3 types of fruit, we'll set "num_classes" to 3
# Be sure to update this number if you plan to have your CNN use a different number of fruits
Y = to_categorical(Y, num_classes=3)


Now, we'll build your Convolutional Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam


# Build our first Convolution layer
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.35))

# Build our 2nd Convolution layer
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.35))

# Dense layers require 1-D datasets, so we'll need to flatten our data
# for this layer
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(3, activation="softmax"))


model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# Validation split is 10%
model.fit(X, Y, epochs=5, validation_split=0.1)




In [ ]:
import cv2
import tensorflow as tf

# MAKE SURE THESE MATCH THE INDEXES OF YOUR CATEGORY
class_names = {0: 'Orange', 1: 'Apple', 2: 'Banana'}

#Model Prediction
import cv2
import numpy as np
def predict_img(image,model):
    test_img=cv2.imread(image)
    test_img=cv2.resize(test_img,(IMG_SIZE,IMG_SIZE))
    test_img=np.expand_dims(test_img, axis=0)
    result=model.predict(test_img)
    r=np.argmax(result)
    print(class_names[r])

predict_img('fruits/apple_test.jpg', model)
predict_img('fruits/banana_test.png', model)
predict_img('fruits/tomato_test.jpg', model)